In [4]:
# import h5py
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from dataclasses import dataclass
from typing import List

In [ ]:
xrootd_url='root://fndca1.fnal.gov:1094/pnfs/fnal.gov/usr/dune/archive/sam_managed_users/jierans/data/7/a/0/b/01558b40-fbc3-4f76-9a35-5681d72e930f-reco_28636553_834.root'

In [5]:
file = ROOT.TFile.Open('output.root')
tree = file.tree
tree.Print()

******************************************************************************
*Tree    :tree      : tree                                                   *
*Entries :       20 : Total =      5519779175 bytes  File  Size =   37042375 *
*        :          : Tree compression factor = 149.02                       *
******************************************************************************
*Br    0 :digit_map : vector<short>                                          *
*Entries :20 : Total  Size= 5519772285 bytes  File Size  = 37038174 *
*Baskets :       20 : Basket Size=   25600000 bytes  Compression= 149.03     *
*............................................................................*
*Br    1 :nu_direction : TVector3                                            *
*Entries :       20 : Total  Size=       1571 bytes  File Size  =        792 *
*Baskets :        2 : Basket Size=      51200 bytes  Compression=   1.33     *
*............................................................

In [48]:
%%timeit

tree.GetEntry(10)
# digit_map = tree.digit_map.data()
digit_map = np.array(tree.digit_map.data(), dtype=np.int32)
correct_shape_map = np.reshape(digit_map, (12, 2560, 4492))
# correct_shape_map = np.ndarray(buffer=digit_map, shape=(12, 2560, 4492), dtype=np.int16)
tf.sparse.from_dense(correct_shape_map)

458 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
a = tf.constant([1, 2, 3, 0, 0], dtype=tf.int16)
tf.sparse.from_dense(a)

UnknownError: JIT compilation failed. [Op:ZerosLike]

2022-05-03 16:43:36.521069: W tensorflow/core/framework/op_kernel.cc:1733] UNKNOWN: JIT compilation failed.


In [50]:
from tqdm import trange, tqdm
from event import Event
import time
dataset = []
for event in tqdm(tree, total=tree.GetEntriesFast()):
    nu_energy = event.nu_energy
    e_energy = event.e_energy
    nu_direction = np.asarray(event.nu_direction)
    e_direction = np.asarray(event.e_direction)
    e_position = np.asarray(event.e_position)
    # write digits
    digit_map = np.array(event.digit_map.data(), dtype=np.int32) # annoyingly tf doesn't work on int16 sometimes...
    plane_map = np.reshape(digit_map, (12, 2560, 4492))
    plane_map = tf.constant(plane_map)
    plane_map = tf.sparse.from_dense(plane_map)
#     for apa in range(12):
#     apa_begin = apa * 2560
#     plane_map.append(np.array(digit_map[apa_begin        : apa_begin + 800]))
#     plane_map.append(np.array(digit_map[apa_begin + 800  : apa_begin + 1600]))
#     plane_map.append(np.array(digit_map[apa_begin + 1600 : apa_begin + 2560]))

    e = Event(nu_energy, 
            e_energy , 
            nu_direction, 
            e_direction, 
            e_position, 
            plane_map)
    dataset.append(e)

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.49it/s]


In [51]:
import pickle
import gzip

with gzip.open('../data/ES_clean.pkl.gz' , 'wb') as f:
    pickle.dump(dataset, f)